Эмоции
01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised



In [1]:
#!pip install librosa
import os
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pathlib

import pylab as pl
import scipy
import scipy.optimize as opt
from sklearn import preprocessing
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import train_test_split

import csv

import tensorflow

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path=input()


/content/drive/MyDrive/audiodataset/Actor_


/content/drive/MyDrive/audiodataset/Actor_

In [ ]:
#Считываем датасет
audiodata=[]
audiodataclass=[]
for i in range(24):
  pathdir=path
  if i<9:
    pathdir+='0'
  for j in os.listdir(pathdir+str(i+1)):
    audiodata.append(librosa.load(pathdir+str(i+1)+'/'+j))
    audiodataclass.append(j.split(sep='-')[2])

In [5]:
#Собираем фичи
def extract_features(y,sr):
    features = []
    #MFCC — это матрица значений, которая захватывает тембральные аспекты.
    features.extend([np.mean(e) for e in librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)])  # mfcc_mean<0..20> средние значения (по 20 значений) из Мел-кепстральных коэффициентов;
    features.extend([np.std(e) for e in librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)])   # mfcc_std стандартные отклонения  (по 20 значений) из Мел-кепстральных коэффициентов;
    # Спектральный центроид - это мера, используемая в цифровой обработке сигналов для характеристики спектр .
    # Он указывает, где расположен центр масс спектра. В восприятии он имеет прочную связь с впечатлением яркости звука.
    features.append(np.mean(librosa.feature.spectral_centroid(y=y, sr=sr).T, axis = 0)[0])     # cent_mean среднее значение из Спектрального центроида;
    features.append(np.std(librosa.feature.spectral_centroid(y=y,sr=sr).T,axis = 0)[0])       # cent_std стандартное отклонение из Спектрального центроида;
    features.append(scipy.stats.skew(librosa.feature.spectral_centroid(y=y,sr=sr).T, axis = 0)[0])    # cent_skew наклон из Спектрального центроида;
    features.append(np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr).T, axis = 0)[0])      # rolloff_mean реднее значение из Спектрального спада;
    features.append(np.std(librosa.feature.spectral_rolloff(y=y, sr=sr).T, axis = 0)[0])       # rolloff_std стандартное отклонение из Спектрального спада;
    return features

In [8]:
#Собираем фичи для каждого
audiodatafitures=[]
for i in range(len(audiodata)):
  audiodatafitures.append(extract_features(audiodata[i][0], audiodata[i][1]))

In [7]:
#дополнительно в файл
audiodatafitures=[]
with open('audiodata.csv', 'w', newline='') as file:
  writer = csv.writer(file, delimiter=',')
  writer.writerow(['data','output'])
  for i in range(len(audiodata)):
    audiodatafitures.append(extract_features(audiodata[i][0], audiodata[i][1]))
    writer.writerows([audiodatafitures[i],str(int(audiodataclass[i]))])


KeyboardInterrupt



In [138]:
#если датасет уже есть
audiodataclass=[]
audiodatafitures=[]
with open('audiodata.csv', 'r', newline='') as csvfile:
  audioreader = csv.reader(csvfile, delimiter=',', quotechar='|')
  counter=0
  for row in audioreader:
    if counter!=0:
      if counter%2==1:
        audiodatafitures.append([float(i) for i in row])
      else:
        audiodataclass.append(int(row[0]))
    counter+=1

In [139]:
#Если есть выбросы
X=preprocessing.StandardScaler().fit(audiodatafitures).transform(audiodatafitures)

In [140]:
#Разбиваем выборку
x_train, x_test, y_train, y_test = train_test_split(X, audiodataclass, test_size=0.2, random_state=10)

In [142]:
#Метод опорных векторов
clf = svm.SVC(kernel='rbf', gamma='auto', C=5)
clf.fit(x_train, y_train)
yhat = clf.predict(x_test)
print("Train set Accuracy: ", metrics.accuracy_score(y_train, clf.predict(x_train)))
print("Test set Accuracy: ",metrics.accuracy_score(y_test, yhat) )

Train set Accuracy:  0.9921875
Test set Accuracy:  0.7291666666666666


Train set Accuracy:  0.9992283950617284

Test set Accuracy:  0.7916666666666666

kernel='rbf', gamma='auto', C=10

Train set Accuracy:  0.9953703703703703

Test set Accuracy:  0.7291666666666666

(kernel='poly', degree=3, C=10)

In [143]:
#Логистическая регрессия
from sklearn.linear_model import LogisticRegression
Log_model = LogisticRegression(multi_class='multinomial', solver='sag', C=5, class_weight='balanced')
Log_model.fit(x_train, y_train)

log_yhat = Log_model.predict(x_test)
# summarize the predicted class
print("Train set Accuracy: ", metrics.accuracy_score(y_train, Log_model.predict(x_train)))
print("Test set Accuracy: ",metrics.accuracy_score(y_test, log_yhat))

Train set Accuracy:  0.6640625
Test set Accuracy:  0.5520833333333334


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Train set Accuracy:  0.6640625

Test set Accuracy:  0.5520833333333334

In [144]:
#CatBoost
#!pip install catboost
from catboost import CatBoostClassifier

Cat_model = CatBoostClassifier(
    iterations=500, depth=4, learning_rate=0.1,
   loss_function='MultiClass', verbose=False)

Cat_model.fit(x_train, y_train)
Cat_yhat=Cat_model.predict(x_test)
# summarize the predicted class
print("Train set Accuracy: ", metrics.accuracy_score(y_train, Cat_model.predict(x_train)))
print("Test set Accuracy: ",metrics.accuracy_score(y_test, Cat_yhat))

Train set Accuracy:  0.9947916666666666
Test set Accuracy:  0.6180555555555556


Train set Accuracy:  0.9891975308641975

Test set Accuracy:  0.6597222222222222


In [145]:
#Метод ближайших соседей
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 4, leaf_size=30, p=1).fit(x_train, y_train)
knn_yhat = knn.predict(x_test)
print("Train set Accuracy: ", metrics.accuracy_score(y_train, knn.predict(x_train)))
print("Test set Accuracy: ",metrics.accuracy_score(y_test, knn_yhat))

Train set Accuracy:  0.8064236111111112
Test set Accuracy:  0.6145833333333334



Train set Accuracy:  0.8310185185185185

Test set Accuracy:  0.5902777777777778



In [147]:
#Решающие деревья
from sklearn.tree import DecisionTreeClassifier

dtree_model = DecisionTreeClassifier(max_depth = 10, criterion='log_loss').fit(x_train, y_train)
dtree_yhat = dtree_model.predict(x_test)
print("Train set Accuracy: ", metrics.accuracy_score(y_train, dtree_model.predict(x_train)))
print("Test set Accuracy: ",metrics.accuracy_score(y_test, dtree_yhat))

Train set Accuracy:  0.953125
Test set Accuracy:  0.3993055555555556


Train set Accuracy:  0.9876543209876543

Test set Accuracy:  0.4236111111111111

In [148]:
#Байес
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB().fit(x_train, y_train)
gnb_yhat = gnb.predict(x_test)
print("Train set Accuracy: ", metrics.accuracy_score(y_train, gnb.predict(x_train)))
print("Test set Accuracy: ",metrics.accuracy_score(y_test, gnb_yhat))

Train set Accuracy:  0.4053819444444444
Test set Accuracy:  0.3819444444444444


Лучше всего подошёл метод опорных векторов из предствленных в sklearn